In [2]:
import pandas as pd
from fuzzywuzzy import fuzz
import numpy as np

/Users/abby/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


This notebooks filters data in the infogroup dataset based upon SIC or NAICS code. It also includes fuzzy string matching to match addresses to the FSIS dataset on meat packing plants.

In [2]:
FSIS_DATA = "../data/fsis-processors-with-location.csv"

INFOGROUP_1997 = ["../data/1997_Business_Academic_QCQ.txt"]
INFOGROUP_1998 = ["../data/1998_Business_Academic_QCQ.txt"]
INFOGROUP_1999 = ["../data/1999_Business_Academic_QCQ.txt"]
INFOGROUP_2000 = ["../data/2000_Business_Academic_QCQ.txt"]
INFOGROUP_2001 = ["../data/2001_Business_Academic_QCQ.txt"]
INFOGROUP_2002 = ["../data/2002_Business_Academic_QCQ.txt"]
INFOGROUP_2003 = ["../data/2003_Business_Academic_QCQ.txt"]
INFOGROUP_2004 = ["../data/2004_Business_Academic_QCQ.txt"]

INFOGROUP_2021 = ["../data/2021_Business_FullFile_QCQ-A.txt", "../data/2021_Business_FullFile_QCQ-B.txt"]
INFOGROUP_2022 = ["../data/2022_Business_Academic_QCQ.txt"]

SMOKE_TEST = False

In [37]:
USEFUL_COLS = [
    "COMPANY",
    "ADDRESS LINE 1",
    "CITY",
    "STATE",
    ]
SIC_COLS = [
    "PRIMARY SIC CODE", 
    'SIC CODE 1', 
    'SIC CODE 2', 
    'SIC CODE 3',
    'SIC CODE 4'
    ]
NAICS_COLS = [
    "NAICS CODE",
    "PRIMARY NAICS CODE"
    ]
DESCRIPTION_COLS = [
    # "NAICS8 DESCRIPTIONS", # only in 2021
    # "SIC6_DESCRIPTIONS (PRIMARYSIC)" # only in 2021
    "SIC6_DESCRIPTIONS (SIC)"
]

In [147]:
USEFUL_COLS = [
    "Company",
    "Address Line 1",
    "City",
    "State",
    ]
SIC_COLS = [
    "Primary SIC Code", 
    'SIC Code 1', 
    'SIC Code 2', 
    'SIC Code 3',
    'SIC Code 4'
    ]
NAICS_COLS = [
    "NAICS Code",
    "Primary NAICS Code"
    ]
DESCRIPTION_COLS = [
    "NAICS8 Descriptions", # only in 2021
    # "SIC6_Description (PRIMARYSIC)" # only in 2021
    "SIC6_Descriptions (SIC)"
]

In [148]:
SIC_CODE = "2015" # Poultry Slaughtering and Processing
NAICS_CODE = "311615" # Poultry Processing

# SIC_CODE = "0251" # Broiler, Fryer, and Roaster Chickens

In [149]:
df_head = pd.read_csv(INFOGROUP_2001[0], nrows=10000, dtype=str)

In [151]:
df_head.columns

Index(['Company', 'Address Line 1', 'City', 'State', 'ZipCode', 'Zip4',
       'County Code', 'Area Code', 'IDCode', 'Location Employee Size Code',
       'Location Sales Volume Code', 'Primary SIC Code', 'SIC6_Descriptions',
       'Primary NAICS Code', 'NAICS8 Descriptions', 'SIC Code',
       'SIC6_Descriptions (SIC)', 'SIC Code 1', 'SIC6_Descriptions (SIC1)',
       'SIC Code 2', 'SIC6_Descriptions(SIC2)', 'SIC Code 3',
       'SIC6_Descriptions(SIC3)', 'SIC Code 4', 'SIC6_Descriptions(SIC4)',
       'Archive Version Year', 'Yellow Page Code',
       'Employee Size (5) - Location', 'Sales Volume (9) - Location',
       'Business Status Code', 'Industry Specific First Byte',
       'Year Established', 'Office Size Code', 'Company Holding Status', 'ABI',
       'Subsidiary Number', 'Parent Number', 'Parent Actual Employee Size',
       'Parent Actual Sales Volume', 'Parent Employee Size Code',
       'Parent Sales Volume Code', 'Site Number', 'Address Type Indicator',
       'Populat

In [152]:
df_head[USEFUL_COLS + SIC_COLS + DESCRIPTION_COLS]

,Company,Address Line 1,City,State,Primary SIC Code,SIC Code 1,SIC Code 2,SIC Code 3,SIC Code 4,NAICS8 Descriptions,SIC6_Descriptions (SIC)
0,COMMUNITY LAUNDROMAT,305 SPRINGFIELD ST,AGAWAM,MA,721101,NaN,NaN,NaN,NaN,DRYCLEANING & LAUNDRY SVCS (EXCEPT COIN-OPERATED),NaN
1,CHRISTY'S PLUMBING & HEATING,54 RAMAH CIR N,AGAWAM,MA,171102,NaN,NaN,NaN,NaN,PLUMBING HTG & AIR-CONDITIONING CONTRACTORS,HEATING CONTRACTORS
2,MECHANICAL PLASTICS INC,PO BOX 406,AGAWAM,MA,308906,NaN,NaN,NaN,NaN,ALL OTHER PLASTICS PRODUCT MANUFACTURING,ASSEMBLY & FABRICATING SERVICE (MFRS)
3,ROSEMARIE'S BEAUTY SALON,PO BOX 217,AGAWAM,MA,723106,NaN,NaN,NaN,NaN,BEAUTY SALONS,NaN
4,U-HAUL CO,1670 MAIN ST,AGAWAM,MA,735910,735934,751903,NaN,NaN,GENERAL RENTAL CENTERS,RENTAL SERVICE-STORES & YARDS
...,...,...,...,...,...,...,...,...,...,...,...
9995,JEMCO CONTRACTING INC,PO BOX 336,LUDLOW,MA,179944,179944,503211,NaN,NaN,OTHER BUILDING EQUIP CONTRACTORS,STONE-CRUSHED
9996,OVERHEAD DOOR,190 MOODY ST,LUDLOW,MA,521102,521102,769983,NaN,NaN,OTHER BUILDING MATERIAL DEALERS,DOORS-GARAGE
9997,WEST SIDE METAL DOOR CORP,190 MOODY ST,LUDLOW,MA,521131,NaN,NaN,NaN,NaN,OTHER BUILDING MATERIAL DEALERS,NaN
9998,CUSTOM MACHINING INDUSTRIES,824 PERIMETER RD,LUDLOW,MA,359903,NaN,NaN,NaN,NaN,MACHINE SHOPS,NaN


In [153]:
def filter_infogroup(filenames: list[str, ...], search_cols: list[str, ...], search_str: str, chunksize: int=10000, smoke_test: bool=SMOKE_TEST):
    filtered_df = pd.DataFrame([])
    for filename in filenames:
        for df in pd.read_csv(filename, iterator=True, chunksize=chunksize):
            rows_to_add = df[df[search_cols].apply(lambda r: r.astype(str).str.contains(search_str, case=False).any(), axis=1)]
            filtered_df = pd.concat([filtered_df, rows_to_add], axis=0)
            if smoke_test:
                break
    return filtered_df

In [154]:
SMOKE_TEST = False
# SIC_CODE = "0259" # Poultry and Eggs, Not Elsewhere Classified
# no results for 0259 or 0251
df_filtered = filter_infogroup(INFOGROUP_2004, SIC_COLS, SIC_CODE, chunksize=1000000)

C:\Users\chacon1\AppData\Local\Temp\ipykernel_3624\3407189000.py:4: DtypeWarning: Columns (42,48) have mixed types. Specify dtype option on import or set low_memory=False.
  for df in pd.read_csv(filename, iterator=True, chunksize=chunksize):
C:\Users\chacon1\AppData\Local\Temp\ipykernel_3624\3407189000.py:4: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  for df in pd.read_csv(filename, iterator=True, chunksize=chunksize):
C:\Users\chacon1\AppData\Local\Temp\ipykernel_3624\3407189000.py:4: DtypeWarning: Columns (28,42) have mixed types. Specify dtype option on import or set low_memory=False.
  for df in pd.read_csv(filename, iterator=True, chunksize=chunksize):
C:\Users\chacon1\AppData\Local\Temp\ipykernel_3624\3407189000.py:4: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  for df in pd.read_csv(filename, iterator=True, chunksize=chunksize):
C:\Users\chacon1\AppData\Local\Tem

In [155]:
df_filtered.head()

,Company,Address Line 1,City,State,ZipCode,Zip4,County Code,Area Code,IDCode,Location Employee Size Code,...,Population Code,Census Tract,Census Block,Latitude,Longitude,Match Code,CBSA Code,CBSA Level,CSA Code,FIPS Code
110199,PHOENIX POULTRY,89 BALDWIN ST,WEST SPRINGFIELD,MA,1089.0,3705.0,13.0,413,2,A,...,5.0,812300.0,4.0,42.095257,-72.619632,P,44140.0,2.0,521.0,25013.0
111791,BENNETT TURKEY FARMS INC,603 MAIN ST,WILBRAHAM,MA,1095.0,1603.0,13.0,413,2,A,...,1.0,813602.0,9.0,42.112605,-72.435751,P,44140.0,2.0,521.0,25013.0
113961,ROYAL HARVEST FOODS,55 AVOCADO ST,SPRINGFIELD,MA,1104.0,3303.0,13.0,413,2,E,...,7.0,800800.0,2.0,42.110191,-72.608206,P,44140.0,2.0,521.0,25013.0
273050,PURITAN FOOD CO,17 FOOD MART RD,BOSTON,MA,2118.0,2801.0,25.0,617,2,D,...,9.0,61200.0,1.0,42.337367,-71.062657,0,14460.0,2.0,148.0,25025.0
292835,MAYFLOWER POULTRY CO,621 CAMBRIDGE ST,CAMBRIDGE,MA,2141.0,1141.0,17.0,617,2,B,...,9.0,352200.0,1.0,42.371989,-71.086675,0,14460.0,2.0,148.0,25017.0


In [156]:
df_filtered.columns

Index(['Company', 'Address Line 1', 'City', 'State', 'ZipCode', 'Zip4',
       'County Code', 'Area Code', 'IDCode', 'Location Employee Size Code',
       'Location Sales Volume Code', 'Primary SIC Code', 'SIC6_Descriptions',
       'Primary NAICS Code', 'NAICS8 Descriptions', 'SIC Code',
       'SIC6_Descriptions (SIC)', 'SIC Code 1', 'SIC6_Descriptions (SIC1)',
       'SIC Code 2', 'SIC6_Descriptions(SIC2)', 'SIC Code 3',
       'SIC6_Descriptions(SIC3)', 'SIC Code 4', 'SIC6_Descriptions(SIC4)',
       'Archive Version Year', 'Yellow Page Code',
       'Employee Size (5) - Location', 'Sales Volume (9) - Location',
       'Business Status Code', 'Industry Specific First Byte',
       'Year Established', 'Office Size Code', 'Company Holding Status', 'ABI',
       'Subsidiary Number', 'Parent Number', 'Parent Actual Employee Size',
       'Parent Actual Sales Volume', 'Parent Employee Size Code',
       'Parent Sales Volume Code', 'Site Number', 'Address Type Indicator',
       'Populat

In [157]:
# save the filtered infogroup data to csv
FILENAME = "../data/poultry_plants_2004.csv"
df_filtered.to_csv(FILENAME)

#### Do Fuzzy String Matching

In [27]:
df_filtered = pd.read_csv("../data/poultry_plants_2022.csv")

In [28]:
df_filtered["Full Address"] = df_filtered["ADDRESS LINE 1"] + ", " + df_filtered["CITY"] + ", " + df_filtered["STATE"] + " " + df_filtered["ZIPCODE"].astype(int).astype(str)

In [29]:
df_filtered["Full Address"] = df_filtered["Full Address"].astype(str)

In [30]:
df_fsis = pd.read_csv(FSIS_DATA, index_col=0)

In [31]:
df_fsis.head()

,EstNumber,EstID,Parent Corporation,Establishment Name,State,Size,Animals Processed,Processed\nVolume\nCategory,Slaughter\nVolume\nCategory,Full Address,latitude,longitude
0,M267,4802,JBS,JBS Tolleson Inc.,AZ,Large,Beef,5.0,4.0,"651 S. 91st Ave, Tolleson, AZ 85353",33.441660,-112.252559
1,M354,5117,Cargill,Cargill Meat Solutions,CA,Large,Beef,5.0,4.0,"3115 S. Fig Ave., Fresno, CA 93706",36.689651,-119.801950
2,M6063A,5289,Central Valley Meat Company,"Central Valley Meat Co., Inc.",CA,Large,Beef,5.0,4.0,"10431 8 3/4 Ave., Hanford, CA 93230",36.321273,-119.612222
3,M783 + P783 + V783,5144,Central Valley Meat Company,Harris Ranch Beef Company,CA,Large,Beef,5.0,4.0,"16277 S McCall Ave., Selma, CA 93662",36.499212,-119.614553
4,M21488 + V21488,8349,Independent,OWB Packers LLC,CA,Large,Beef,4.0,4.0,"57 East Shank Road, Brawley, CA 92227",33.000969,-115.521786


In [32]:
df_poultry = df_fsis[df_fsis["Animals Processed"].str.contains("Chicken")].copy()

In [33]:
df_poultry["Sales Volume (Location)"] = np.NaN

In [34]:
len(df_poultry)

258

In [35]:
df_poultry.head()

,EstNumber,EstID,Parent Corporation,Establishment Name,State,Size,Animals Processed,Processed\nVolume\nCategory,Slaughter\nVolume\nCategory,Full Address,latitude,longitude,Sales Volume (Location)
71,P1317 + V1317,4495,Cargill,Wayne Farms LLC,AL,Large,Chicken,5.0,5.0,"700 McDonald Avenue, Albertville, AL 35950",34.260726,-86.203222,NaN
72,P7485 + V7485,4518,Cargill,Wayne Farms LLC,AL,Large,Chicken,5.0,5.0,"1020 County Road 114, Jack, AL 36346",31.500628,-85.903438,NaN
73,P912,4493,Cargill,"Wayne Farms, LLC",AL,Large,Chicken,5.0,5.0,"444 Baskin Street South, Union Springs, AL 36089",32.139166,-85.721704,NaN
74,P7342 + V7342,4516,Cargill,Wayne Farms LLC,AL,Large,Chicken,5.0,5.0,"808 Ross Clark Circle NE, Dothan, AL 36303",31.225754,-85.362068,NaN
75,P1235,4509,Cargill,Wayne Farms LLC,AL,Large,Chicken,5.0,5.0,"254 Ipsco Road, Decatur, AL 35601",34.607601,-87.042595,NaN


In [36]:
df_match = pd.DataFrame()
df_match["Sales Volume (Location)"] = np.NaN

In [37]:
plants_to_update = {}
for i, fsis in df_poultry.iterrows():
    fsis_address = fsis["Full Address"].lower()
    for k, infogroup in df_filtered.iterrows():
        infogroup_address = infogroup["Full Address"].lower()
        if fuzz.token_sort_ratio(infogroup_address, fsis_address) > 75:
            print(f"Found a match at index {k}")
            print(infogroup_address)
            print(fsis_address)
            # plants_to_update[i] = infogroup['SALES VOLUME (9) - LOCATION']
            df_poultry.loc[i, "Sales Volume (Location)"] = infogroup['SALES VOLUME (9) - LOCATION']
            break

Found a match at index 641
700 mcdonald ave, albertville, al 35950
700 mcdonald avenue, albertville, al 35950
Found a match at index 253
1020 county road 114, jack, al 36346
1020 county road 114, jack, al 36346
Found a match at index 705
444 baskin st s, union springs, al 36089
444 baskin street  south, union springs, al 36089
Found a match at index 240
2045 highway 244, russellville, al 35654
2045 highway 244, russellville, al 35654
Found a match at index 583
3500 lake guntersville park dr, guntersville, al 35976
3500 lake guntersville park drive, guntersville, al 35976
Found a match at index 262
4693 county road 636, enterprise, al 36330
4693 county road 636, enterprise, al 36330
Found a match at index 637
764 george cagle dr, collinsville, al 35961
764 george cagle drive, collinsville, al 35961
Found a match at index 561
3301 3rd ave s, jasper, al 35501
3301 3rd avenue, jasper, al 35501
Found a match at index 635
57 melvin clark rd, eufaula, al 36027
57 melvin clark road, baker hill

In [38]:
df_poultry.head()

,EstNumber,EstID,Parent Corporation,Establishment Name,State,Size,Animals Processed,Processed\nVolume\nCategory,Slaughter\nVolume\nCategory,Full Address,latitude,longitude,Sales Volume (Location)
71,P1317 + V1317,4495,Cargill,Wayne Farms LLC,AL,Large,Chicken,5.0,5.0,"700 McDonald Avenue, Albertville, AL 35950",34.260726,-86.203222,438268.0
72,P7485 + V7485,4518,Cargill,Wayne Farms LLC,AL,Large,Chicken,5.0,5.0,"1020 County Road 114, Jack, AL 36346",31.500628,-85.903438,576660.0
73,P912,4493,Cargill,"Wayne Farms, LLC",AL,Large,Chicken,5.0,5.0,"444 Baskin Street South, Union Springs, AL 36089",32.139166,-85.721704,271713.0
74,P7342 + V7342,4516,Cargill,Wayne Farms LLC,AL,Large,Chicken,5.0,5.0,"808 Ross Clark Circle NE, Dothan, AL 36303",31.225754,-85.362068,NaN
75,P1235,4509,Cargill,Wayne Farms LLC,AL,Large,Chicken,5.0,5.0,"254 Ipsco Road, Decatur, AL 35601",34.607601,-87.042595,NaN


In [39]:
df_poultry["Sales Volume (Location)"].isna().sum()

139

In [40]:
# save poultry plants with matches sales data
df_poultry.to_csv("../data/poultry_plants_with_sales.csv")